## Serpent Input Generator Demo
1. Showcase of functionality from the in progress development of the serpentInputGenerator

In [6]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.data.materialLibrary import MATLIB

## Using pregenerated materials for effeciency
1. users can take advantadge of the pregenerated material library and tweek paramters to their liking
2. users can also use data from pregenerated materials and create their own
3. users can access all the pregenerated materials using the ``list()`` function on the material library ``MATLIB``

In [7]:
list(MATLIB)

['UO2', 'Zr', 'boron', 'lightWater', 'plenum']

## Creating our custom UO2 fuel
1. using the ``vars()`` function we can see the default pregenerated UO2 material parameters 
2. for this example we will be generating a higher enrichement uranium oxide


In [9]:
vars(MATLIB['UO2'])

{'name': 'UO2',
 'isBurn': True,
 'isModer': True,
 'dens': -10.2141,
 'nuclides': array([92235, 92238,  8016], dtype=int64),
 'fractions': array([0.00081508, 0.02218898, 0.04600811]),
 'temp': 0.0,
 'xsLib': '09c',
 'modLib': 'None',
 'color': 'orange'}

In [21]:
fuel = MATLIB['UO2']
fuel.set('fractions', np.array([0.001331338626, 0.02218898, 0.04600811])) # 6% Enrichment
fuel.set('xsLib', "12c") #Higher enrichment cores will run at higher temperatures so we shall change the xs library suffix
fuel.set('color','red') #change the color of the fuel for plotting purposes
vars(fuel) #Display updated properties of our new fuel

{'name': 'UO2',
 'isBurn': True,
 'isModer': True,
 'dens': -10.2141,
 'nuclides': array([92235, 92238,  8016], dtype=int64),
 'fractions': array([0.00133134, 0.02218898, 0.04600811]),
 'temp': 0.0,
 'xsLib': '12c',
 'modLib': 'None',
 'color': 'red'}

## Creating a pin using our new fuel
1. a pin element can be created using user created or pre generated materials
2. the user will have to specify the materials and outer radius of each material in an order dependent fashion
2. For this scenario the pin will have 3 regions conisiting of fuel, cladding, and moderator
3. the user will use the newly created fuel and is content using the pregenerated Zr cladding and lightWater moderator


In [24]:
fuelPin = pin("fuelPin", 3) #id = fuelPin, numberOfRegions = 3
vars(fuelPin)

{'id': 'fuelPin',
 'nregions': 3,
 'materials': array([], dtype=object),
 'radii': array([], dtype=float64)}

In [33]:
fuelPin.set('materials', np.array([fuel, MATLIB['Zr'], MATLIB['lightWater']]))
fuelPin.set('radii', np.array([.45, .47]))
print(fuelPin.toString()) #displaythe pin element in serpent format 

pin fuelPin
UO2	0.45
Zr	0.47
lightWater


